In [2]:
import glob
import os
import librosa
import numpy as np
import pandas as pd
import speechpy
from matplotlib import pyplot as plt
from keras import layers, models
from keras.callbacks import TensorBoard
from keras.models import Sequential, model_from_json
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

mfccs_df = pd.read_csv("mfccs_features.csv")
allfeat_df = pd.read_csv("all_features.csv")
features = allfeat_df.loc[:, allfeat_df.columns != "labels"].values
mfccs_features = mfccs_df.loc[:, mfccs_df.columns != "labels"].values
labels = allfeat_df.loc[:, "labels"].values

Using TensorFlow backend.


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.2, random_state=42)
model = Sequential()
model.add(Dense(52, kernel_initializer='uniform'))
model.add(Dense(26, kernel_initializer='uniform', activation='relu'))
model.add(Dense(13, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
# adam = Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
tensorboard = TensorBoard(log_dir='./tb-logs/4-layer/all/run3_noreluinput_sigmoid', 
                          write_graph=True, 
                          write_images=False)
model.fit(X_train, y_train,
          epochs=200,
          validation_data=(X_test, y_test),
          shuffle=True,
          verbose=False,
          callbacks=[tensorboard])

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    mfccs_features, labels, test_size=0.2, random_state=42)
model = Sequential()
model.add(Dense(52, kernel_initializer='uniform'))
model.add(Dense(26, kernel_initializer='uniform', activation='relu'))
model.add(Dense(13, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))
# adam = Adam(lr=0.0001)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
tensorboard = TensorBoard(log_dir='./tb-logs/4-layer/mfccs/run1', 
                          write_graph=True, 
                          write_images=False)
model.fit(X_train, y_train,
          epochs=200,
          validation_data=(X_test, y_test),
          shuffle=True,
          verbose=False,
          callbacks=[tensorboard])

In [6]:
scores = model.evaluate(X_test, y_test)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

25/25 [==============================] - 0s 372us/step
acc: 84.00%
